In [1]:
from datasets import load_dataset
data = load_dataset('arbml/EASC')

Found cached dataset parquet (/home/magedsaeed/.cache/huggingface/datasets/arbml___parquet/arbml--EASC-85636dd59f0cb935/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/1 [00:00<?, ?it/s]

In [2]:
data

DatasetDict({
    train: Dataset({
        features: ['article', 'summary'],
        num_rows: 153
    })
})

In [3]:
def truncate_examples(example):
    example['article'] = example['article'][:4290]
    return example
data = data.map(truncate_examples)

Loading cached processed dataset at /home/magedsaeed/.cache/huggingface/datasets/arbml___parquet/arbml--EASC-85636dd59f0cb935/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-9349a87a0df06ace.arrow


In [4]:
max_sample_index = 0
max_sample_length = 0
for i,example in enumerate(data['train']):
    if len(example['article'])+len(example['summary']) > max_sample_length:
        max_sample_length = len(example['article'])+len(example['summary'])
        max_sample_index = i
max_sample_length,max_sample_index
    

(7717, 7)

In [5]:
sys_msg = "Summarize the following statement to Arabic"
def create_chat_prompt(sys_msg, input_text):
    return [
        {"role": "system", "content": sys_msg}, 
        {"role": "user", "content": input_text}
    ]

def create_chat_example(article, summary):
    return [
        {"role": "system", "content": article, "name": "example_user"},
        {"role": "system", "content": summary, "name": "example_assistant"},
    ]

In [6]:
!mkdir -p ../registry/data/easc

In [7]:
# dev_df = split_data['train'].to_pandas()
# dev_df["sample"] = dev_df.apply(lambda x: create_chat_example(x['article'], x['summary']), axis=1)
# dev_df[["sample"]].to_json('../registry/data/summarization/few_shot.jsonl', lines=True, orient="records")

test_df = data['train'].to_pandas()
test_df["input"] = test_df['article'].apply(lambda x: create_chat_prompt(sys_msg, x))
test_df["ideal"] = test_df['summary']
test_df[["input", "ideal"]].to_json('../registry/data/easc/samples.jsonl', lines=True, orient="records")

In [8]:
test_df

,article,summary,input,ideal
0,﻿لودفيج فان بيتهوفن مؤلف موسيقي ألماني ولد عام...,له الفضل الاعظم في تطوير الموسيقى الكلاسيكية....,"[{'role': 'system', 'content': 'Summarize the ...",له الفضل الاعظم في تطوير الموسيقى الكلاسيكية....
1,﻿لوتشانو بافاروتي مواليد مودينا في 12 أكتوبر 1...,لوتشانو بافاروتي مواليد مودينا في 12 اكتوبر 19...,"[{'role': 'system', 'content': 'Summarize the ...",لوتشانو بافاروتي مواليد مودينا في 12 اكتوبر 19...
2,﻿السياسة البربرية تعد هذه السياسة من بين النقط...,ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاس...,"[{'role': 'system', 'content': 'Summarize the ...",ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاس...
3,﻿حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دا...,﻿حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دا...,"[{'role': 'system', 'content': 'Summarize the ...",﻿حرب اليمن أو حرب شمال اليمن الأهلية هي حرب دا...
4,﻿حرب فيتنام أو الحرب الهندوصينية الثانية كانت ...,وقد عُرف هؤلاء بالفيت كونغ.\nورويداً رويداً تش...,"[{'role': 'system', 'content': 'Summarize the ...",وقد عُرف هؤلاء بالفيت كونغ.\nورويداً رويداً تش...
...,...,...,...,...
148,﻿أعلنت مصادر رسمية أن الحكومة الانتقالية برئاس...,اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...,"[{'role': 'system', 'content': 'Summarize the ...",اعلنت مصادر رسمية ان الحكومة الانتقالية برئاسة...
149,﻿ ردت القوى السياسية المصرية بعنف على قرار الب...,﻿ وكانت حالة من الارتباك والانشقاق داخلصفوف ال...,"[{'role': 'system', 'content': 'Summarize the ...",﻿ وكانت حالة من الارتباك والانشقاق داخلصفوف ال...
150,﻿ قالت مصادر في حركة فتح كبرى الفصائل الفلسطين...,"وبحسب هذه المصادر فان ""قادة الحركة في قطاع غز...","[{'role': 'system', 'content': 'Summarize the ...","وبحسب هذه المصادر فان ""قادة الحركة في قطاع غز..."
151,﻿قضية لوكربي هي قضية جنائية ترتبت على سقوط طائ...,اصرت الدولتان على طلبهما ورفضت ليبيا الاستجاب...,"[{'role': 'system', 'content': 'Summarize the ...",اصرت الدولتان على طلبهما ورفضت ليبيا الاستجاب...


In [9]:
import os
os.environ["OPENAI_API_KEY"] = "<openai-key>"
os.environ["EVALS_THREADS"] = "3"
os.environ["EVALS_THREAD_TIMEOUT"] = "600"

In [10]:
summarization_task_specs = """
easc:
    id: easc.test.v1
    metrics: [average_rouge1,average_rougeL]
    description: Evaluate Arabic text summarization
# Define the eval
easc.test.v1:
  # Specify the class name as a dotted path to the module and class
  class: evals.elsuite.summarization:Summarization
  args:
    samples_jsonl: easc/samples.jsonl
    num_few_shot: 0 # max few shots to use

""".strip()
with open("../registry/evals/easc.yaml", "w") as file:
    file.write(summarization_task_specs)

In [13]:
# !oaieval gpt-3.5-turbo summarization --max_samples 1
record_path = "../../evals/eval_results/easc.jsonl"

!oaieval gpt-3.5-turbo-0301 easc --record_path {record_path} --seed 41 --modelspec_extra_options temperature=0.0

[2023-05-14 18:56:41,356] [registry.py:156] Loading registry from /home/magedsaeed/MyProjects/evals/evals/registry/evals
[2023-05-14 18:56:41,468] [registry.py:156] Loading registry from /home/magedsaeed/.evals/evals
[2023-05-14 18:56:42,777] [oaieval.py:213] Run started: 230514155642Q63TRH2B
[2023-05-14 18:56:42,912] [data.py:75] Fetching easc/samples.jsonl
[2023-05-14 18:56:42,923] [eval.py:32] Evaluating 153 samples
[2023-05-14 18:56:42,926] [eval.py:152] Running in threaded mode with 3 threads!
  3%|█▍                                         | 5/153 [00:56<26:09, 10.60s/it][2023-05-14 18:57:39,485] [_common.py:105] Backing off openai_chat_completion_create_retrying(...) for 1.3s (openai.error.RateLimitError: That model is currently overloaded with other requests. You can retry your request, or contact us through our help center at help.openai.com if the error persists. (Please include the request ID 1861c56b95df2b2330f6b7e83b18c08b in your message.))
[2023-05-14 18:57:51,992] [__in

In [14]:
sorted([len(article)+len(summary) for article,summary in zip(data['train']['article'],data['train']['summary'])])

[770,
 1015,
 1024,
 1041,
 1048,
 1071,
 1094,
 1094,
 1201,
 1254,
 1269,
 1291,
 1323,
 1385,
 1386,
 1401,
 1404,
 1431,
 1456,
 1517,
 1564,
 1619,
 1634,
 1641,
 1646,
 1662,
 1673,
 1674,
 1760,
 1773,
 1781,
 1820,
 1832,
 1868,
 1872,
 1948,
 1965,
 1979,
 1993,
 2001,
 2005,
 2023,
 2042,
 2049,
 2055,
 2060,
 2075,
 2126,
 2138,
 2226,
 2227,
 2294,
 2295,
 2319,
 2337,
 2464,
 2465,
 2476,
 2477,
 2479,
 2497,
 2501,
 2522,
 2529,
 2531,
 2561,
 2593,
 2607,
 2622,
 2647,
 2648,
 2673,
 2701,
 2722,
 2723,
 2753,
 2753,
 2782,
 2810,
 2834,
 2837,
 2843,
 2878,
 2901,
 2918,
 2945,
 2956,
 2964,
 2967,
 2989,
 2992,
 2994,
 3024,
 3042,
 3126,
 3133,
 3215,
 3246,
 3260,
 3313,
 3332,
 3335,
 3336,
 3401,
 3449,
 3473,
 3485,
 3517,
 3526,
 3615,
 3653,
 3667,
 3678,
 3730,
 3745,
 3754,
 3789,
 3815,
 3929,
 4078,
 4129,
 4131,
 4155,
 4372,
 4373,
 4397,
 4416,
 4421,
 4466,
 4503,
 4504,
 4510,
 4522,
 4563,
 4567,
 4572,
 4795,
 4825,
 4859,
 4946,
 5033,
 5142,
 5210,


In [17]:
import pandas as pd
events = record_path

with open(events, "r") as f:
    events_df = pd.read_json(f, lines=True)

In [18]:
for i, r in pd.json_normalize(events_df[events_df.type == "sampling"].data).iterrows():
    print(f"Prompt: {r.prompt[-1]}")
    print(f"Sampled: {r.sampled}")
    print(f"{i}","--" * 25)

Prompt: {'role': 'user', 'content': '\ufeffالسياسة البربرية تعد هذه السياسة من بين النقط الأساسية التي اعتمدتها الكتابات الاستعمارية ، ويمكن تحديد مرحلتين في تعامل الكتابات الفرنسية مع هذه القضية . فخلال أواخر القرن التاسع عشر والعقد الأول من القرن العشرين اتجه اهتمام المفكرين الفرنسيين إلى الاهتمام بعلاقات البربر بالعرب عبر التاريخ وخاصة في العصور الإسلامية الأولى محاولين الإيحاء بأن البربر لم يقبلوا أبدا العرب " الدخلاء " ، فكأنما جاء الفرنسيون إذن بتخليصهم من الدخيل وربطهم بالأصل ، إذ في هذه الفترة انتشرت النظرية القائلة بأن البربر ينحدرون من أصول أوربية . كما راج استعمال أفكار ابن خلدون لفظ " الأعراب "بمعنى العرب ، إذ وجد فيه الفرنسيون مبررا لادعاءاتهم القائلة بأن المغرب عرف دائما انتشار الفوضى بوجود العرب . وبما أن ابن خلدون منهم فقد جعله الفرنسيون ذريعة قوية للقول بأنه لا حل لهذه الفوضى إلا بإلحاق الفرع بالأصل أي ربط البربر ( الفرع ) بأوربا ( الأصل ) . ومع ذلك فقد بقيت هذه الكتابات طيلة القرن التاسع عشر والعقد الأول من القرن العشرين رائجة على مستوى الأفكار فقط . وبمجيء ليوطي حاول